In [ ]:
import bootstrap
from torch.utils.tensorboard import SummaryWriter
import MinkowskiEngine as ME
from torch import nn
import torch
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline
mpl.rcParams['figure.figsize'] = [8, 6]
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.grid'] = True

## Tell pytorch we have a GPU if we do
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.device(device)

SEED=12345
_=np.random.seed(SEED)
_=torch.manual_seed(SEED)
writer = SummaryWriter("log")

In [ ]:
## Use the common dataset loader
from core.data.datasets import paired_2d_dataset_ME, triple_ME_collate_fn
from core.analysis.image_utils import make_dense, make_dense_from_tensor, make_dense_array

In [ ]:
import h5py
from scipy.sparse import coo_matrix

## This function just pulls an image directly from the file, without going through a pytorch dataloder
## You would need to have a file open: f = h5py.File(input_file, 'r')
def show_image(i, f):
    group = f[str(i)]
    data = group['data'][:]
    row = group['row'][:]
    col = group['col'][:]

    ## Use the format that ME requires                                                                                                                                                                         
	## Note that we can't build the sparse tensor here because ME uses some sort of global indexing                                                                                                            
	## And this function is replicated * num_workers                                                                                                                                                           
    this_sparse = coo_matrix((data, (row, col)), dtype=np.float32, shape=(800, 256))    
    this_image = this_sparse.toarray()

    gr = plt.imshow(this_image, origin='lower')
    plt.colorbar(gr)
    plt.show()

In [ ]:
import torchvision.transforms.v2 as transforms
from datasets.fsd.augmentations_2d import get_transform

# Force reload so I can play with changes outside jupyter...
import importlib
import core.data.augmentations_2d
importlib.reload(core.data.augmentations_2d)
import core.data.augmentations_2d as aug

## 768 is chosen to fit with the current encoder architecture
x_max=256
y_max=768

x_orig=256
y_orig=800

x_max = x_orig
y_max = y_orig

mod_transform = transforms.Compose([
    	    aug.RandomBlockZeroImproved([50,100], [5,10], [0,x_orig], [0,y_orig]),
            aug.RandomBlockZeroImproved([500,2000], [1,3], [0,x_orig], [0,y_orig]),
            aug.RandomInPlaceHorizontalFlip(),
            aug.RandomInPlaceVerticalFlip(),
    	    aug.RandomHorizontalFlip(x_max=x_orig),
            aug.RandomVerticalFlip(y_max=y_orig),
            aug.RandomPixelNoise2D(10),
            aug.UnlogCharge(),
            aug.GridJitter(),
            aug.SplitJitterCoords(10),
            aug.RandomShear2D(0.1, 0.1),
            aug.RandomRotation2D(6),
            aug.RandomStretch2D(0.1, 0.1),
    	    aug.RandomGridDistortion2D(100, 5, 2, 25),
    	    aug.RandomScaleCharge(0.05),
        	aug.RandomJitterCharge(0.05),
    	    aug.BilinearSplatMod(0.3, 0.5, 0.5),
            aug.RelogCharge(),
       	    aug.RandomScaleCharge(0.02),
        	aug.RandomJitterCharge(0.02),
            aug.SemiRandomCrop(x_max, y_max, 20),
            ])

aug_transform = mod_transform #
# aug_transform = get_transform('fsd', "vbigaugbilinfixnostretch")

## Load some images into a data loader
sim_dir = "/pscratch/sd/c/cwilk/FSD/SIMULATIONv2"
data_dir = "/pscratch/sd/c/cwilk/FSD/DATA"
nom_transform = transforms.Compose([
            aug.FirstRegionCrop((800, 256), (768, 256)),
            ])

sim_dataset = paired_2d_dataset_ME(sim_dir, nom_transform=nom_transform, aug_transform=aug_transform, max_events=100000)
data_dataset = paired_2d_dataset_ME(data_dir, nom_transform=nom_transform, aug_transform=aug_transform, max_events=100000)
print("Found", data_dataset.__len__(), "data events")

In [ ]:
def set_ops(A, B):
    both = np.array([x for x in A if any((B == x).all(1))])
    only_A = np.array([x for x in A if not any((B == x).all(1))])
    only_B = np.array([x for x in B if not any((A == x).all(1))])

    return both, only_A, only_B

In [ ]:
import numpy as np
import time

def profile_transforms_avg(dataset, transform_block, n_events=10):
    times_accum = {t.__class__.__name__: [] for t in transform_block.transforms}

    for idx in range(n_events):
        _, _, _, _, coords, feats = dataset[idx]
        for t in transform_block.transforms:
            start = time.perf_counter()
            coords, feats = t(coords, feats)
            end = time.perf_counter()
            times_accum[t.__class__.__name__].append(end - start)

    times_avg = {k: np.mean(v) for k, v in times_accum.items()}
    return times_avg

In [ ]:
## BilinearSplat starts at 8.911 ms averaged over 1000 events
times = profile_transforms_avg(data_dataset, mod_transform, n_events=1000)
times_sorted = sorted(times.items(), key=lambda x: x[1], reverse=True)

for name, t in times_sorted:
    print(f"{name:30s}: {t*1000:.3f} ms") 

In [ ]:
def simple_test(dataset, n=0):

    aug1_bcoords, aug1_bfeats, aug2_bcoords, aug2_bfeats, orig_bcoords, orig_bfeats = dataset[n]

    print("AUG1 #1:", aug1_bcoords[0], aug1_bfeats[0])
    print("AUG2 #1:", aug2_bcoords[0], aug2_bfeats[0])
    print("ORIG #1:", orig_bcoords[0], orig_bfeats[0])

    print("AUG1 SHAPE:", aug1_bcoords.shape, aug1_bfeats.shape)
    print("AUG2 SHAPE:", aug2_bcoords.shape, aug2_bfeats.shape)    
    print("ORIG SHAPE:", orig_bcoords.shape, orig_bfeats.shape)

    ## Bilinear splat should really have a threshod of ~0.5 to be close to the data distribution...
    # print(orig_bfeats.min(), orig_bfeats.max())
    both, only_A, only_B = set_ops(orig_bcoords, aug1_bcoords)
    print(len(both), len(only_A), len(only_B))
    

In [ ]:
simple_test(sim_dataset, 69861)

In [ ]:
## Visualise data
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

def make_aug_comp_plot(dataset, ids=[0], save_name=None):

    cmap = cm.turbo.copy()
    cmap.set_under("#F0F0F0")

    # Visualize the image
    n_augs = 5
    n_ids = len(ids)
    indices = np.arange(n_augs*n_ids)

    ## Set up the canvas
    plt.figure(figsize=(n_augs*2.1, n_ids*6))

    ## To keep track of the subplot
    running_n = 0
    
    ## Loop over images
    for i in ids:
    
        ## The dataset works with pairs, so this is just a bit hacky to get more examples
        aug1_bcoords, aug1_bfeats, aug2_bcoords, aug2_bfeats, orig_bcoords, orig_bfeats = dataset[i]
        aug3_bcoords, aug3_bfeats, aug4_bcoords, aug4_bfeats, _, _ = dataset[i]

        ## Keep the same scale for them all
        #orig_max = max(orig_bfeats)
        
        augs = [make_dense_array(orig_bcoords, orig_bfeats.squeeze(), y_max, x_max),
                make_dense_array(aug1_bcoords, aug1_bfeats.squeeze(), y_max, x_max),
                make_dense_array(aug2_bcoords, aug2_bfeats.squeeze(), y_max, x_max),
                make_dense_array(aug3_bcoords, aug3_bfeats.squeeze(), y_max, x_max),
                make_dense_array(aug4_bcoords, aug4_bfeats.squeeze(), y_max, x_max)]

        orig_max = augs[0].max()
        orig = augs[0]
        print("orig_min =", np.min(orig[orig != 0]))

        
        ## Loop over augmentations
        for aug in augs:
            ax = plt.subplot(n_ids,n_augs,running_n+1)
            # mean_val = np.mean(aug[np.isfinite(aug)]) 
            mean_val = np.mean(aug[aug != 0])
            print(running_n, mean_val)
            nonzero_vals = aug[aug > 0]
            vmax = np.percentile(nonzero_vals, 80)
            ax.imshow(aug, origin='lower', cmap=cmap, vmin=1e-6, vmax=vmax) #, vmax=orig_max)
            ax.axis('off')
            running_n += 1
    plt.tight_layout()
    if save_name: plt.savefig(save_name, dpi=300, bbox_inches='tight')
    plt.show()
    # plt.close()

In [ ]:
import numpy as np
def find_biggest_events(dataset, ret_num=1):

    size_list = []
    
    max_event = 5000 #len(dataset)
    for n in range(max_event):
        if n%1000==0:
            print("Processed", n, "/", max_event)
        _, _, _, _, _, orig_bfeats = dataset[n]
        size_list.append(orig_bfeats.shape[0])

    indices = np.argsort(size_list)[::-1]  # reverse for descending order
    return indices[:ret_num]

In [ ]:
big_list = find_biggest_events(data_dataset, 50)

In [ ]:
print(big_list)

In [ ]:
## Visualise data
def make_aug_diff_plot(dataset, images=[0]):

    ## The dataset works with pairs, so this is just a bit hacky to get more examples
    aug1_bcoords, aug1_bfeats, aug2_bcoords, aug2_bfeats, orig_bcoords, orig_bfeats = dataset[n]
    aug3_bcoords, aug3_bfeats, aug4_bcoords, aug4_bfeats, _, _ = dataset[n]

    nom_dense  = make_dense_array(orig_bcoords, orig_bfeats.squeeze(), 800, 256)
    aug1_dense = make_dense_array(aug1_bcoords, aug1_bfeats.squeeze(), 800, 256)
    aug2_dense = make_dense_array(aug2_bcoords, aug2_bfeats.squeeze(), 800, 256)
    aug3_dense = make_dense_array(aug3_bcoords, aug3_bfeats.squeeze(), 800, 256)
    aug4_dense = make_dense_array(aug4_bcoords, aug4_bfeats.squeeze(), 800, 256)

    diff1 = aug1_dense - nom_dense
    diff2 = aug2_dense - nom_dense
    diff3 = aug3_dense - nom_dense
    diff4 = aug4_dense - nom_dense

    augs = [nom_dense, diff1, diff2, diff3, diff4]
    
    vmax = max(np.max(np.abs(img)) for img in images)
    vmin = -vmax  # symmetric around zero
    vmin = -1
    vmax = 1

    ntotal = len(augs)
    nimages = len(images)
    plt.figure(figsize=(ntotal*1.8,6*nimages))
    for i, aug in enumerate(augs, 1):
        ax = plt.subplot(nimages, naugs, i)
        im = ax.imshow(aug, origin='lower', cmap='seismic', vmin=vmin, vmax=vmax)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.tight_layout()
    plt.show()

In [ ]:
## Dump a few events!
## Interesting numbers: 3, 28, 50, 75, 77, 95, 179, 237, 239, 272, 303, 347, 69861, 73664, 16498
test_list = [16498, 179, 69861]
make_aug_comp_plot(data_dataset, test_list, save_name="example_augmentations.jpg")